In [1]:
!pip install  "mlflow-skinny" "ray[serve]" 

  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached pydantic_core-2.33.2-cp311-cp311-win_amd64.whl.metadata (6.9 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/120.0 kB ? eta -:--:--
   ---------------------------------------- 120.0/120.0 kB 6.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/65.6 kB ? eta -:--:--
   ---------------------------------------- 65.6/65.6 kB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/201.6 kB ? eta -:--:--
   --------------------------------------- 201.6/201.6 kB 12.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   -------- ------------------------------- 0.4/1.8 MB 7.6 MB/s eta 0:00:01
   ------------------- -------------------- 0.9/1.8 MB 9.3 MB/s eta 0:00:01
   ---------------------------------------  1.8/1.8 MB 12.8 MB/s eta 0:00:01
   ------------------------------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lightning 2.0.9 requires pydantic<2.2.0,>=1.7.4, but you have pydantic 2.11.7 which is incompatible.
opentelemetry-exporter-otlp-proto-grpc 1.28.2 requires opentelemetry-exporter-otlp-proto-common==1.28.2, but you have opentelemetry-exporter-otlp-proto-common 1.28.0 which is incompatible.
opentelemetry-exporter-otlp-proto-grpc 1.28.2 requires opentelemetry-proto==1.28.2, but you have opentelemetry-proto 1.28.0 which is incompatible.
opentelemetry-exporter-otlp-proto-grpc 1.28.2 requires opentelemetry-sdk~=1.28.2, but you have opentelemetry-sdk 1.36.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.28.0 requires opentelemetry-sdk~=1.28.0, but you have opentelemetry-sdk 1.36.0 which is incompatible.
opentelemetry-instrumentation 0.49b0 requires opentelemetry-seman

In [ ]:
import mlflow
from transformers import pipeline


class MyTranslationModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        self.lang_from = context.model_config.get("lang_from", "en")
        self.lang_to = context.model_config.get("lang_to", "de")

        self.input_label: str = context.model_config.get("input_label", "prompt")

        self.model_ref: str = context.model_config.get("hfhub_name", "google-t5/t5-base")

        self.pipeline = pipeline(
            f"translation_{self.lang_from}_to_{self.lang_to}",
            self.model_ref,
        )

    def predict(self, context, model_input, params=None):
        prompt = model_input[self.input_label].tolist()

        return self.pipeline(prompt)

c:\Users\Kavitha padala\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


c:\Users\Kavitha padala\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\Kavitha padala\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\Kavitha padala\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlflow\pyfunc\utils\data_validation.py:134: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


: 

In [ ]:
import pandas as pd

with mlflow.start_run():
    model_info = mlflow.pyfunc.log_model(
        "translation_model",
        registered_model_name="translation_model",
        python_model=MyTranslationModel(),
        pip_requirements=["transformers"],
        input_example=pd.DataFrame(
            {
                "prompt": ["Hello my name is Jonathan."],
            }
        ),
        model_config={
            "hfhub_name": "google-t5/t5-base",
            "lang_from": "en",
            "lang_to": "de",
        },
    )

2025/08/20 08:17:08 INFO mlflow.pyfunc: Inferring model signature from input example
c:\Users\Kavitha padala\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

c:\Users\Kavitha padala\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Kavitha padala\.cache\huggingface\hub\models--google-t5--t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\Kavitha padala\AppData\Local\Programs\Python\Python311\Lib\site-packa

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
en_to_de_version: str = str(model_info.registered_model_version)

In [ ]:
print(model_info.signature)